In [19]:
import pandas as pd
from pathlib import Path
import AppHelpers as ah

In [65]:
zipDB = pd.read_csv(Path('Resources/zip_code_database.csv'))
# create a new dataframe with only the columns we need with no duplicates or index
# zipDB = zipDB[['zip', 'primary_city', 'state', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)

# get the city and zip code from a given lat/lon using zipCodeDBData
def get_city_zip(lat, lon):
   for zipCode in zipDB:
        # trim the lat/lon to 2 decimal places
        lat = round(lat, 2)
        lon = round(lon, 2)
        if zipCode[3] == lat and zipCode[4] == lon:
           return zipCode[1], zipCode[0]
   return None, None


# get the city and zip code from a given lat/lon using zipCodeDBData
def get_cityzip(lat, lon):
   # trim the lat/lon to 2 decimal places and convert to float64
   # lat = f"{lat:.2f}"
   # lon = f"{lon:.2f}"
   newlat = float(str(lat)[:str(lat).find('.')+3]) 
   newlon = float(str(lon)[:str(lon).find('.')+3])
   # print(f"lat: {lat} -> {newlat}, lon: {lon} -> {newlon}")
   # zdata = zipDB[(zipDB['latitude'] == lat) & (zipDB['longitude'] == lon)]
   zdata = zipDB.query('latitude == @newlat & longitude == @newlon')
   if len(zdata) > 0:
      return zdata["primary_city"].values[0], zdata["zip"].values[0]
   return None, None
# # filter zipDB where state == NY
# nydata = zipDB[zipDB['state'] == 'NY']

# nydata.head(None)


In [66]:
# test the function
# data = get_cityzip(round(40.7287448,2),round(-74.0321082,2))
data = get_cityzip(40.7287448,-74.0321082)
# data = get_cityzip(40.72,-74.03)
print(data)

# # query city from zipDB where latitute = 40.72 and longitude = -74.03
# nydata = zipDB[(zipDB['latitude'] == 40.72) & (zipDB['longitude'] == -74.03)]
# city = nydata['primary_city'].values[0]
# city

('Jersey City', 7311)


In [67]:
jan_data_path = Path("Data/JC-202001-citibike-tripdata.csv")
feb_data_path = Path("Data/JC-202002-citibike-tripdata.csv")
mar_data_path = Path("Data/JC-202003-citibike-tripdata.csv")
apr_data_path = Path("Data/JC-202004-citibike-tripdata.csv")
may_data_path = Path("Data/JC-202005-citibike-tripdata.csv")

jan_data = pd.read_csv(jan_data_path)
feb_data = pd.read_csv(feb_data_path)
mar_data = pd.read_csv(mar_data_path)
apr_data = pd.read_csv(apr_data_path)
may_data = pd.read_csv(may_data_path)

# Combine all data into a single DataFrame
combined_data = pd.concat([feb_data, mar_data, apr_data, may_data], ignore_index=True)

# add month column as first column and name it based on the month in the starttime column
combined_data.insert(0, "Month", combined_data['starttime'].str[5:7])
# convert month column value to full month name
combined_data['Month'] = combined_data['Month'].replace(['02', '03', '04', '05'], ['Feb', 'Mar', 'Apr', 'May'])

combined_data.insert(1, "City","")
combined_data.insert(2, "ZipCode","")

In [69]:
# slice the dataframe to get the first 100 rows
combined_data = combined_data.iloc[0:50]
print(len(combined_data))

# write a loop to add city name and zip code to the dataframe using get_city_zip
for index, row in combined_data.iterrows():
    cityzip = get_cityzip(row["start station latitude"], row["start station longitude"])
    combined_data.loc[index, "City"] = cityzip[0]
    combined_data.loc[index, "ZipCode"] = cityzip[1]

# # # write a loop to add city name and zip code to the dataframe
# # for index, row in combined_data.iterrows():
# #     cityzip = ah.get_city_zip(row["start station latitude"], row["start station longitude"])
# #     combined_data.loc[index, "City"] = cityzip[0]
# #     combined_data.loc[index, "ZipCode"] = cityzip[1]

combined_data.head(None)

50


,Month,City,ZipCode,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,Feb,Jersey City,7311,841,2020-02-01 00:02:13.7650,2020-02-01 00:16:15.4720,3199,Newport Pkwy,40.728745,-74.032108,3213,Van Vorst Park,40.718489,-74.047727,42217,Subscriber,1980,1
1,Feb,None,None,206,2020-02-01 00:16:11.3850,2020-02-01 00:19:37.4460,3273,Manila & 1st,40.721651,-74.042884,3639,Harborside,40.719252,-74.034234,42160,Subscriber,1990,1
2,Feb,None,None,859,2020-02-01 00:23:22.1090,2020-02-01 00:37:41.5060,3185,City Hall,40.717732,-74.043845,3268,Lafayette Park,40.713464,-74.062859,42204,Customer,1991,1
3,Feb,None,None,693,2020-02-01 00:25:14.5170,2020-02-01 00:36:48.0280,3185,City Hall,40.717732,-74.043845,3277,Communipaw & Berry Lane,40.714358,-74.066611,42165,Subscriber,1987,1
4,Feb,None,None,1473,2020-02-01 00:32:30.5050,2020-02-01 00:57:04.3740,3192,Liberty Light Rail,40.711242,-74.055701,3639,Harborside,40.719252,-74.034234,26156,Customer,1969,0
5,Feb,Jersey City,7311,129,2020-02-01 00:34:14.4380,2020-02-01 00:36:23.5540,3202,Newport PATH,40.727224,-74.033759,3199,Newport Pkwy,40.728745,-74.032108,29302,Subscriber,1992,2
6,Feb,None,None,394,2020-02-01 00:36:33.0060,2020-02-01 00:43:07.4130,3201,Dey St,40.737711,-74.066921,3281,Leonard Gordon Park,40.745910,-74.057271,42241,Subscriber,1969,1
7,Feb,Jersey City,7311,279,2020-02-01 00:52:29.4100,2020-02-01 00:57:08.5210,3187,Warren St,40.721124,-74.038051,3211,Newark Ave,40.721525,-74.046305,42184,Subscriber,1987,1
8,Feb,None,None,242,2020-02-01 01:11:13.0100,2020-02-01 01:15:15.3820,3203,Hamilton Park,40.727596,-74.044247,3213,Van Vorst Park,40.718489,-74.047727,42254,Subscriber,1989,1
9,Feb,None,None,451,2020-02-01 01:16:04.9460,2020-02-01 01:23:36.4580,3213,Van Vorst Park,40.718489,-74.047727,3268,Lafayette Park,40.713464,-74.062859,42217,Subscriber,1999,1
